## Car Evaluation Data [(source link)](https://archive.ics.uci.edu/dataset/19/car+evaluation)

In [1]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
car_evaluation = fetch_ucirepo(id=19) 
  
# data (as pandas dataframes) 
X = car_evaluation.data.features 
y = car_evaluation.data.targets 
  
# metadata 
print(car_evaluation.metadata) 
  
# variable information 
print(car_evaluation.variables) 

{'uci_id': 19, 'name': 'Car Evaluation', 'repository_url': 'https://archive.ics.uci.edu/dataset/19/car+evaluation', 'data_url': 'https://archive.ics.uci.edu/static/public/19/data.csv', 'abstract': 'Derived from simple hierarchical decision model, this database may be useful for testing constructive induction and structure discovery methods.', 'area': 'Other', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 1728, 'num_features': 6, 'feature_types': ['Categorical'], 'demographics': [], 'target_col': ['class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1988, 'last_updated': 'Thu Aug 10 2023', 'dataset_doi': '10.24432/C5JP48', 'creators': ['Marko Bohanec'], 'intro_paper': {'ID': 249, 'type': 'NATIVE', 'title': 'Knowledge acquisition and explanation for multi-attribute decision making', 'authors': 'M. Bohanec, V. Rajkovič', 'venue': '8th Intl Workshop on Expert Systems and their Applications, 

In [68]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio

from IPython.display import display, Markdown

plt.style.use("dark_background")
pio.templates.default = "plotly_dark"

In [24]:
data = pd.concat([X, y], axis=1)
data.rename(columns={'class': 'acceptance'}, inplace=True)
data['acceptance_class'] = data['acceptance'].replace({'good': 'acc', 'vgood': 'acc'})

buying = 'buying'
maint = 'maint'
doors = 'doors'
persons = 'persons'
lug_boot = 'lug_boot'
safety = 'safety'
acceptance = 'acceptance'
acceptance_class = 'acceptance_class'

order = ['low', 'med', 'high', 'vhigh']

data.head()

,buying,maint,doors,persons,lug_boot,safety,acceptance,acceptance_class
0,vhigh,vhigh,2,2,small,low,unacc,unacc
1,vhigh,vhigh,2,2,small,med,unacc,unacc
2,vhigh,vhigh,2,2,small,high,unacc,unacc
3,vhigh,vhigh,2,2,med,low,unacc,unacc
4,vhigh,vhigh,2,2,med,med,unacc,unacc


In [4]:
for column in data.columns:
    print(f"* {column}: {data[column].unique()}")

* buying: ['vhigh' 'high' 'med' 'low']
* maint: ['vhigh' 'high' 'med' 'low']
* doors: ['2' '3' '4' '5more']
* persons: ['2' '4' 'more']
* lug_boot: ['small' 'med' 'big']
* safety: ['low' 'med' 'high']
* acceptance: ['unacc' 'acc' 'vgood' 'good']
* acceptance_class: ['unacc' 'acc']


In [31]:
acceptance_proportion = data[acceptance_class].value_counts().reset_index()
acceptance_proportion.columns = ['acceptance', 'count']

px.pie(
    acceptance_proportion,
    names=acceptance,
    values='count',
    color=acceptance,
    color_discrete_sequence=['red', 'green'],
    height=500,
    width=500,
)

In [66]:
buying_acceptance = data.pivot_table(
    acceptance,
    index=buying,
    columns=acceptance_class,
    aggfunc='count'
).loc[order]

px.bar(
    buying_acceptance,
    x=buying_acceptance.index,
    y=['acc', 'unacc'],
    barmode='group',
    color_discrete_sequence=['green', 'red']
)

In [95]:
persons_acceptance = data.pivot_table(
    acceptance,
    index=persons,
    columns=acceptance_class,
    aggfunc='count',
    fill_value=0
)

px.bar(
    persons_acceptance,
    x=persons_acceptance.index,
    y=persons_acceptance.columns,
    barmode='stack',
    color_discrete_sequence=['green', 'red']
)

In [126]:
safety_acceptance = data.pivot_table(
    acceptance,
    index=safety,
    columns=acceptance_class,
    aggfunc='count',
    fill_value=0
).loc[order[:-1]]

px.bar(
    safety_acceptance,
    x=safety_acceptance.index,
    y=safety_acceptance.columns,
    barmode='group',
    color_discrete_sequence=['green', 'red']
)

In [148]:
import plotly.express as px

# Create the pivot table
pivot_df = data.pivot_table(
    values=acceptance,          # what you're counting
    index=[safety, buying],     # row grouping
    columns=acceptance_class,   # category columns
    aggfunc='count',
    fill_value=0
).reset_index()                 # reset for Plotly compatibility

# Melt into long form for Plotly Express
pivot_long = pivot_df.melt(
    id_vars=[safety, buying],
    value_name='count',
    var_name=acceptance_class
)

# Plot
fig = px.bar(
    pivot_long,
    x=buying,
    y='count',
    color=acceptance_class,
    barmode='stack',
    facet_col=safety,  # separate panels by safety level
    color_discrete_sequence=['green', 'red'],
    title='Acceptance Counts by Safety and Buying'
)

fig.update_layout(
    template='plotly_dark',  # optional dark theme
    xaxis_title='Buying',
    yaxis_title='Count',
    legend_title='Acceptance Class'
)

fig.show()


In [146]:
lug_boot_acceptance = data.pivot_table(
    acceptance,
    index=lug_boot,
    columns=acceptance_class,
    aggfunc='count'
).loc[['small', 'med', 'big']]

px.bar(
    lug_boot_acceptance,
    x=lug_boot_acceptance.index,
    y=lug_boot_acceptance.columns,
    barmode='stack',
    color_discrete_sequence=['green', 'red']
)

In [170]:
data.pivot_table(
    acceptance,
    index=[persons, safety, lug_boot, buying],
    columns=acceptance_class,
    aggfunc='count',
    fill_value=0,
).loc['4']

acceptance_class        acc  unacc
safety lug_boot buying            
high   big      high     12      4
                low      16      0
                med      16      0
                vhigh     8      8
       med      high     12      4
                low      16      0
                med      16      0
                vhigh     8      8
       small    high     12      4
                low      16      0
                med      16      0
                vhigh     8      8
low    big      high      0     16
                low       0     16
                med       0     16
                vhigh     0     16
       med      high      0     16
                low       0     16
                med       0     16
                vhigh     0     16
       small    high      0     16
                low       0     16
                med       0     16
                vhigh     0     16
med    big      high     12      4
                low      16      0
                med      16      0
                vhigh     8      8
       med      high      6     10
                low      14      2
                med      12      4
                vhigh     4     12
       small    high      0     16
                low      12      4
                med       8      8
                vhigh     0     16

In [164]:
data

,buying,maint,doors,persons,lug_boot,safety,acceptance,acceptance_class
0,vhigh,vhigh,2,2,small,low,unacc,unacc
1,vhigh,vhigh,2,2,small,med,unacc,unacc
2,vhigh,vhigh,2,2,small,high,unacc,unacc
3,vhigh,vhigh,2,2,med,low,unacc,unacc
4,vhigh,vhigh,2,2,med,med,unacc,unacc
...,...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good,acc
1724,low,low,5more,more,med,high,vgood,acc
1725,low,low,5more,more,big,low,unacc,unacc
1726,low,low,5more,more,big,med,good,acc
